In [24]:
!sudo apt install -q tesseract-ocr
!cp "../models/spa.traineddata" /usr/share/tesseract-ocr/4.00/tessdata/
!ls -l /usr/share/tesseract-ocr/4.00/tessdata/

total 32180
drwxr-xr-x 2 root root     4096 Mar 18 16:18 configs
-rw-r--r-- 1 root root  4113088 Sep 15  2017 eng.traineddata
-rw-r--r-- 1 root root 10562727 Sep 15  2017 osd.traineddata
-rw-r--r-- 1 root root      572 Apr  7  2018 pdf.ttf
-rw------- 1 root root 18256019 Apr  3 06:44 spa.traineddata
drwxr-xr-x 2 root root     4096 Mar 18 16:18 tessconfigs


In [0]:
!pip install -q pytesseract
# !pip install -q fitz
!pip install -q PyMuPDF

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Proyectos_Ley_Congreso/notebooks")

In [2]:
import cv2
import pytesseract
import fitz
import pytesseract
from PIL import Image
import io
import glob
import re
import os
from os.path import join, basename
import time
from datetime import datetime
import pandas as pd
pytesseract.get_tesseract_version()

LooseVersion ('4.0.0-beta.1')

In [0]:
lotes=[                                                              "201608","2016109","201610","201611","201612"
"201701","201702","201703","201704","201705","201706","201707","201708","2017109","201710","201711","201712"
"201801","201802","201803","201804","201805","201806","201807","201808","2018109","201810","201811","201812"
"201901","201902","201903","201904","201905","201906","201907","201908","2019109"]

In [26]:
for i in lotes:
  rutaEntrada = "../data/raw/pdf/"
  rutaSalida = "../data/intermediated/pdf_recognized/"
  patron = "*{}*.pdf".format(i) ## nombre del archivo a procesar
  flist=glob.glob(join(rutaEntrada,patron))
  start_time = time.time()
  archivos=[]
  fechas=[]
  paginas=[]
  textos=[]
  for f in flist:
      print("Procesando archivo %s" % f)
      try:
          doc = fitz.open(f)
          for i in range(len(doc)):
              aux=[]
              for img in doc.getPageImageList(i):
                  xref=img[0]
                  stream = io.BytesIO(fitz.Pixmap(doc,xref).getPNGData())
                  imagenPag = Image.open(stream)### Obtener las imágenes
                  try:
                      info=pytesseract.image_to_osd(imagenPag, lang='spa')
                      angulo = int(re.search('(?<=Rotate: )\d+', info).group(0))
                      if angulo != 0:
                          infotext="El angulo a rotar es: %s" % (angulo-180)
                          print(infotext)
                          imagenPag = imagenPag.rotate(angulo-180, expand=True)
                      aux.append(pytesseract.image_to_string(imagenPag, lang='spa'))            
                  except:
                      continue
              archivos.append(basename(f))
              fechas.append(datetime.strptime(f[-12:-4],'%Y%m%d'))
              paginas.append(i+1)
              textos.append(" ".join(aux)) 
      except:
          print("no se puede abrir el archivo {}".format(f))
          continue
  print("El tiempo transcurrido es %f segundos" % (time.time() - start_time))
  data={"Archivo_Origen":archivos,
      "Fecha":fechas,
      "Pagina":paginas,
      "Texto":textos}
  data=pd.DataFrame(data)
  print(data.shape)
  data.to_pickle(rutaSalida+"{}.pkl".format(patron[1:7]))

(66, 4)


,Archivo_Origen,Fecha,Pagina,Texto
0,PL0083920161230.pdf,2016-12-30,1,"o eA [N U\n\nN Ministerio ,\n' de Relaciones E..."
1,PL0083920161230.pdf,2016-12-30,2,"— Informe (DGT) N* 063-2016, de fecha 25 de no..."
2,PL0083920161230.pdf,2016-12-30,3,"CONGRESO DE LA REPÚBLICA\nLima, ..2Z......de....."
3,PL0083920161230.pdf,2016-12-30,4,"R esolución ©5¿/WZC¿ p2-20é-ne\n\nLima, 16 de ..."
4,PL0083920161230.pdf,2016-12-30,5,Carpeta de perfeccionamiento del Acuerdo entre...
